

![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/CLASSIFICATION_TR_NEWS.ipynb)




# **Classify Turkish News**

## 1. Colab Setup

In [ ]:
# Install PySpark and Spark NLP
! pip install -q pyspark==3.3.0 spark-nlp==4.2.8

In [2]:
import json
import pandas as pd
import numpy as np

import sparknlp
import pyspark.sql.functions as F

from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
from sparknlp.annotator import *
from sparknlp.base import *
from sparknlp.pretrained import PretrainedPipeline
from pyspark.sql.types import StringType, IntegerType

## 2. Start Spark Session

In [3]:
spark = sparknlp.start()

print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)

spark

Spark NLP version 4.2.8
Apache Spark version: 3.3.0


## 3. Select the DL model and re-run all the cells below

In [4]:
model_name = "classifierdl_bert_news"

## 4. Some sample examples

In [5]:
text_list = [
"""JSL Erzurumgücü Futbol Şube Sorumlusu Ahmet Mehmet, yaklaşan genel kurul öncesinde açıklamalarda bulundu. JSL Erzurumgücü Kulübü'nün 20 Mayıs perşembe günü yapılacak olağan genel kurulunda başkanlığa kesinlikle aday olmayacağını ifade eden Ahmet Mehmet, yaşanan süreç ile ilgili detayları anlattı.""",
"""Tek kadınlarda elde ettiği 23 zaferle Margeret Court'un ardından en fazla grand slam turnuvası kazanan tenisçi konumundaki 39 yaşındaki Williams, tek erkeklerde dünyanın en iyisi olarak Federer'i gördüğünü söyledi.""",
"""AB Komisyonu, üye ülkelerin çok uluslu şirketlere farklı vergi uygulamalarını önlemek amacıyla hazırladığı yol haritasını yayımladı. Buna göre, AB ülkelerinde verimli ve adil kurumlar vergisi sistemi kurulması teşvik edilecek.""",
"""Bugün ABD Kongresi’nde yapacağı konuşma metnine göre, Yellen Biden hükümetinin önerdiği paketin şirketlerin net karlılılığını ve küresel rekabet gücünü artıracağını belirterek “umuyoruz ki iş dünyası liderleri bunu göz önünde bulundurarak İstihdam Planı’nı destekler” ifadesini kullanacak.""",
"""Karadeniz Teknik Üniversitesinden (KTÜ) yapılan açıklamaya göre, 14 yaşındaki kız çocuğu, sırtında ve belinde uzun zamandır var olan ağrı, dengesizlik, vücudunda şekil ve duruş bozukluğu şikayetleri dolayısıyla KTÜ Farabi Hastanesine başvurdu.""",
"""Koronavirüs bazı kişilerde ağır belirtiler verirken, bazı kişilerde de belirti göstermiyor. Popular Science'de yer alan habere göre İngiltere'de son yapılan araştırma, belirtsiz korona geçirenlerin şüphelerini gidermek için önemli ipuçları verdi.""",
"""24 Mayıs-21 Haziran tarihleri arasında yapılacak ve Covid-19 salgını nedeniyle bu yıl kısıtlı sayıda izleyicinin ağırlanacağı 22. Sabancı Uluslararası Adana Tiyatro Festivali'nde 18 oyun sahnelenecek. Festivalin biletleri online olarak satışa çıktı.""",
"""Karadeniz’de üzerinde insan yerleşimi bulunmuş nadir adalardan olan Giresun Adası’nda Eylül ve Ekim aylarında arkeolog ve sanat tarihçilerinde oluşan uzman ekip tarafından kurtarma kazısı yapılacak.""",
"""Yapay zekanın yaşadığı iletişim sorunlarını çözmek isteyen Google, LaMDA isimli yapay zeka algoritmasını 18 Mayıs’ta düzenlenen I/O etkinliğinde tanıttı.""",
"""Türk geliştiricilerin oyunu 50 milyondan fazla indirme ile dünyanın en çok indirilen mobil oyunu oldu. 2021 yılının Ocak ayında kurulan Türk oyun şirketi Gargamel Games’in ilk oyunu Bridge Face,  Nisan ayında dünyada en çok indirilen mobil oyun olmayı başardı.""",
"""Cumhurbaşkanı Yardımcısı Sarı Selim, Meclis'te grubu bulunan partilere "Anayasayı uzlaşmayla hazırlayalım" çağrısı yaptı. Selim muhalefetin uzlaşmaya yanaşmaması halinde, 2023'te kendi çalışmalarıyla millete gideceklerini söyledi.""",
"""Güzellikler Partisi Genel Başkanı Seval Şeker JSLTV televizyonunda yaptığı söyleşide erken seçime hazır olduklarını söyledi. Adayın henüz belirlenmemiş olmasının bir çelişki yaratmadığını savundu."""
]

## 5. Define Spark NLP pipeline

In [7]:
document = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

embeddings = BertSentenceEmbeddings\
    .pretrained('labse', 'xx') \
    .setInputCols(["document"])\
    .setOutputCol("sentence_embeddings")

document_classifier = ClassifierDLModel.pretrained(model_name, "tr") \
    .setInputCols(["sentence_embeddings"]) \
    .setOutputCol("class_")    
     
nlpPipeline = Pipeline(
    stages=[
      document, 
      embeddings,
      document_classifier
      ])

labse download started this may take some time.
Approximate size to download 1.7 GB
[OK!]
classifierdl_bert_news download started this may take some time.
Approximate size to download 22 MB
[OK!]


## 6. Run the pipeline

In [8]:
df = spark.createDataFrame(text_list, StringType()).toDF("text")
result = nlpPipeline.fit(df).transform(df)

## 7. Visualize results

In [9]:
result.select(F.explode(F.arrays_zip(result.document.result, 
                                     result.class_.result)).alias("cols")) \
      .select(F.expr("cols['0']").alias("document"),
              F.expr("cols['1']").alias("class")).show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------+
|document                                                                                                                                                                                                                                                                                                 |class    |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------+
|JSL Erzurumgücü Futbol Şube Sorumlusu Ahmet Mehmet, yaklaşan genel ku